### NDBI to Geotiff
Created by Amos Bennett.<br>
Last Updated 12 Jun 20.

In [ ]:
import sys
import datacube
from datacube.helpers import write_geotiff
from datacube.utils import cog
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("/home/554/ab4513/dea-notebooks/Scripts")
from dea_datahandling import load_ard
from dea_plotting import rgb
from dea_plotting import display_map
from dea_bandindices import calculate_indices

In [ ]:
print(datacube.__version__)

In [ ]:
dc = datacube.Datacube(app="NDBI")

In [ ]:
# Set the central latitude and longitude
central_lat = -28.1904
central_lon = 152.8880

# Beaudesert 152.8880°E 28.1904°S

# Set the buffer to load around the central coordinates (even numbers such as 0.2, 1.0, 2.2 etc)
buffer = 0.6

# Compute the bounding box for the study area
study_area_lat = (central_lat - buffer, central_lat + buffer)
study_area_lon = (central_lon - buffer, central_lon + buffer)

display_map(x=study_area_lon, y=study_area_lat, margin=-0.2)

In [ ]:
import shapely
from shapely.geometry import Point

# Create a range for generating point grid.
magic_number = int(buffer*10/1)
rng = range(int(magic_number/2))

x_coord = []
y_coord = []

# Calculate x and y for point grid.
for i in rng:
    l = i/5 + 0.1
    neg_lat = central_lat - l
    neg_lon = central_lon - l
    pos_lat = central_lat + l
    pos_lon = central_lon + l
    x_coord.append(neg_lon)
    x_coord.append(pos_lon)
    y_coord.append(neg_lat)
    y_coord.append(pos_lat)
    
coords = []

# Create list of shapely points.
for x in x_coord:
    for y in y_coord:
        p = Point(x, y)
        coords.append(p)

In [ ]:
# Fire event date
todays_date = '2020-02-29'

# Length of baseline period
baseline_length = '3 months'

In [ ]:
# Define dates for loading data
if baseline_length == '12 months':
    time_step = timedelta(days=365)
if baseline_length == '6 months':
    time_step = timedelta(days=182.5)
if baseline_length == '3 months':
    time_step = timedelta(days=91)

# Calculate the start and end date for baseline data load
start_date = datetime.strftime(((datetime.strptime(todays_date, '%Y-%m-%d'))-time_step), '%Y-%m-%d')
end_date = datetime.strftime(((datetime.strptime(todays_date, '%Y-%m-%d'))-timedelta(days=1)), '%Y-%m-%d')

In [ ]:
# Print dates
print('start_date:  '+start_date)
print('end_date:    '+end_date)

In [ ]:
def NDBI_processing(coordinates):
    # Load all data in basline period avalible from ARD data
    baseline_ard = load_ard(dc=dc,
                   products=['s2a_ard_granule', 's2b_ard_granule'],
                   x = (coordinates.x - 0.1, coordinates.x + 0.1),
                   y = (coordinates.y - 0.1, coordinates.y + 0.1),
                   time=(start_date, end_date),
                   measurements=['nbart_swir_2',
                                 'nbart_nir_1'],
                   #min_gooddata=0.1,
                   output_crs='EPSG:32756',
                   resolution=(-10,10),
                   group_by='solar_day')

    # Find the earliest date from which an observation has been returned from NRT collection
    last_ard_date = (pd.to_datetime((baseline_ard.time[-1]).values)).strftime('%Y-%m-%d')

    # Load remaining data in baseline period available from ARD collection
    baseline_nrt = load_ard(dc=dc,
                   products=['s2a_nrt_granule', 's2b_nrt_granule'],
                   x = (coordinates.x - 0.1, coordinates.x + 0.1),
                   y = (coordinates.y - 0.1, coordinates.y + 0.1),
                   time=(last_ard_date,end_date),
                   measurements=['nbart_swir_2',
                                 'nbart_nir_1'],
                   #min_gooddata=0.1,
                   output_crs='EPSG:32756',
                   resolution=(-10,10),
                   group_by='solar_day')

    # Remove milliseconds 
    baseline_ard['time'] = baseline_ard.indexes['time'].normalize()
    baseline_nrt['time'] = baseline_nrt.indexes['time'].normalize()

    # Find duplicates
    duplicate_dates = baseline_nrt['time'].isin(baseline_ard['time'].values)

    # Only select NRT that is not duplicated in ARD
    baseline_nrt = baseline_nrt.sel(time=~duplicate_dates)

    # Concantenate NRT and ARD data together 
    baseline_combined = xr.concat([baseline_ard, baseline_nrt], dim='time').sortby('time')

    del baseline_ard
    del baseline_nrt

    # Calculate NDBI for the baseline images
    baseline_combined = calculate_indices(baseline_combined, 
                                          index='NDBI', 
                                          collection='ga_s2_1', 
                                          drop=False)

    # Compute median using all observations in the dataset along the time axis
    baseline_image = baseline_combined.median(dim='time')

    del baseline_combined

    # Select NDBI
    baseline_NDBI = baseline_image.NDBI

    del baseline_image

    # Turn NDBI into a Xarray Dataset for export to GeoTIFF
    ndbi_dataset = baseline_NDBI.to_dataset(name='ndbi')
    write_geotiff(f'./geotiffs/{coordinates.x}_{coordinates.y}_NDBI.tif', ndbi_dataset)

    del baseline_NDBI
    del ndbi_dataset

In [ ]:
# Iterate through all shapely points to generate a NDBI geotiff.
for i in coords:
    NDBI_processing(i)